## 집들이에 태깅된 상품정보 스크래핑

해당 코드는 100개에 한번씩 데이터를 저장하는 방식으로 짜여져 있습니다.

먼저 파일을 저장할 폴더를 새로 만들고 그 경로를 `path`에 넣어주세요

In [1]:
path = '경로' ##### 저장할 폴더의 경로. 슬래쉬(/)로 끝난다면 마지막 슬래쉬를 지워주세요.

상품 상세페이지가 적혀있는 파일을 불러옵니다

In [2]:
import pandas as pd
df = pd.read_csv('prd_urls.csv')
df = df.drop('Unnamed: 0', axis = 1)
df = df.reset_index(drop = True)
df

,id,상품url
0,0,https://ohou.se/productions/538782/selling
1,0,https://ohou.se/productions/803775/selling
2,0,https://ohou.se/productions/1068049/selling
3,0,https://ohou.se/productions/580038/selling
4,0,https://ohou.se/productions/961600/selling
...,...,...
243075,4691,https://ohou.se/productions/154011/selling
243076,4691,https://ohou.se/productions/224590/selling
243077,4691,https://ohou.se/productions/224582/selling
243078,4691,https://ohou.se/productions/224584/selling


본인이 맡은부분을 start와 end에 넣어주세요.

`김민지`  [11300:45300]

`김예원`  [45300:79300]

`박유정`  [79300:113300]

`이소현`  [113300:147300]

`이지혜`  [147300:181300]

`최수연`  [181300:215300]

`송나현`  [:11300], [215300:]

In [3]:
start =  ##### 자신이 맡은 부분 시작점
end =  ##### 자신이 맡은 부분 종료지점

In [4]:
df_prd = pd.DataFrame(columns = {'브랜드명', '상품명', '상품가격', '카테고리', '리뷰수', '스크랩수', '스타일링샷수'})
df_prd = pd.concat([df[start:end], df_prd], axis = 1)
df_prd = df_prd[['id', '상품url', '브랜드명', '상품명', '상품가격', '카테고리', '리뷰수', '스크랩수', '스타일링샷수']]
df_prd

,id,상품url,브랜드명,상품명,상품가격,카테고리,리뷰수,스크랩수,스타일링샷수
0,0,https://ohou.se/productions/538782/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,https://ohou.se/productions/803775/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,https://ohou.se/productions/1068049/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,https://ohou.se/productions/580038/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,https://ohou.se/productions/961600/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,https://ohou.se/productions/357692/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,https://ohou.se/productions/1137934/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,https://ohou.se/productions/1006108/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,https://ohou.se/productions/1438532/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,https://ohou.se/productions/1539603/selling,NaN,NaN,NaN,NaN,NaN,NaN,NaN


아래 코드로 스크래핑을 시작합니다. 

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display
import warnings

In [ ]:
# 스크래핑할 창 열기
warnings.filterwarnings(action='ignore')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿모드로 열기
driver = webdriver.Chrome()
# 페이지가 모두 로드될 때까지 3초 기다려주도록 처리
driver.implicitly_wait(3)

In [ ]:
for idx in df_prd.index:

    driver.get(df_prd['상품url'][idx])
    content = driver.find_element(By.CLASS_NAME, 'production-selling-overview.container')

    product = content.find_element(By.CLASS_NAME, 'production-selling-header__title')
    price = content.find_element(By.CLASS_NAME, 'production-selling-header__price__price').find_element(By.CLASS_NAME, 'number')
    categories = content.find_elements(By.CLASS_NAME, 'commerce-category-breadcrumb__entry')
    cates = []
    for cts in categories:
        cates.append(cts.text)


    df_prd['카테고리'][idx] = cates
    df_prd['브랜드명'][idx] = product.find_element(By.TAG_NAME, 'a').text
    df_prd['상품명'][idx] = product.find_element(By.CLASS_NAME, 'production-selling-header__title__name').text

    if price:
        df_prd['상품가격'][idx] = price.text

    df_prd['스크랩수'][idx] = driver.find_element(By.CLASS_NAME, 'production-selling-header__action').text
    for name in driver.find_elements(By.CLASS_NAME, 'production-selling-section__title'):
        if '스타일링' in name.text:
            df_prd['스타일링샷수'][idx] = name.find_element(By.CLASS_NAME, 'count').text
        if '리뷰' in name.text:
            df_prd['리뷰수'][idx] = name.find_element(By.CLASS_NAME, 'count').text

    if idx % 100 == 99:
        print(idx + 1)
        df_prd.loc[start:idx+1].to_csv(path + f'/prd_data_{start}_{idx}.csv')

driver.close()

df_prd.to_csv(path + f'/prd_data_{start}_{idx}.csv')


- 만약 코드를 돌리다 끊어질 경우
    - 괜찮아요! 100개 단위로 저장되어 있으니 가장 마지막에 저장된 파일 제목에 힌트가 있습니다.
    - 마지막 파일의 파일명이 `prd_data_0_399.csv` 라면 `start`에 400을 입력하고 다시 돌리면 됩니다.
    - 돌리기 전에 마지막 파일을 제외하고 다 삭제해주세요! 파일들을 모아서 병합해야하는데, 이전에 파일들을 모두 가지고 있으면 중복값이 생깁니다.
    - 400부터 다시 돌리면 `prd_data_400_799.csv` 이런식으로 바로 파일명이 연결될 거예요 이렇게 연결되는 파일을 모아서 주시면 됩니다.
    
- 돌아가다 끊긴 게 아니라 뭔가 오류가 생긴 경우
    - 바로 저에게 연락주세요! 1만개까지는 생기는 오류를 잡았는데 또 어떤 오류가 나타날지 모르겠어요 😢
    
**데이터가 많아서 오래 걸릴텐데 함께 돌려주시는 여러분 정말 감사해요♥️**

**그럼 잘 부탁드립니다☺️**